In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./9.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(input_.loc[a, 'Url'],
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except:
                    continue

            # = = = = = = = = = = = = = = =
            
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0]
            
            # = = = = = = = = = = = = = = =

            i, dict_src = 0, {}
            list_src = list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
            for src in list_src:
                if src.startswith('http'):
                    dict_src[str(i)] = src
                    i += 1
                    
            # = = = = = = = = = = = = = = =

            part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
            
            # = = = = = = = = = = = = = = =

            description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
            
            # = = = = = = = = = = = = = = =

            application_summary = '\n'.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip()

            # = = = = = = = = = = = = = = =

            brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()
            
            # = = = = = = = = = = = = = = =

            vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
            vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

            # = = = = = = = = = = = = = = =

            dict_specification = {}
            list_tr = html.xpath('//section[@id="productSpec"]/div/table/tr')
            for i, tr in zip(range(len(list_tr)), list_tr):
                dict_specification[str(i)] = {tr.xpath('./th/text()')[0].strip().removesuffix(':').strip(): tr.xpath('./td/text()')[0].strip()}

            # = = = = = = = = = = = = = = =

            list_oem = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
            oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Type': type_,
                                     'Part Number': part_number,
                                     'Brand': brand,
                                     'Description': description,
                                     'Application Summary': application_summary,
                                     'OEM': oem,
                                     'Url': input_.loc[a, 'Url'],
                                     'Vehicle Url': vehicle_url,
                                     'Json_Src': json.dumps(dict_src),
                                     'Json_Specification': json.dumps(dict_specification)}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./10.part_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./part_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：1] - 96444.https://www.dormanproducts.com/p-24595-700-367.aspx
[剩余数量：1000] - [当前时间：14:21:47]

[状态：ok，尝试次数：1] - 96445.https://www.dormanproducts.com/p-18728-901-087.aspx
[剩余数量：999] - [当前时间：14:21:48]

[状态：ok，尝试次数：1] - 96446.https://www.dormanproducts.com/p-49887-901-101.aspx
[剩余数量：998] - [当前时间：14:21:48]

[状态：ok，尝试次数：1] - 96432.https://www.dormanproducts.com/p-53058-955-1477.aspx
[剩余数量：997] - [当前时间：14:21:48]

[状态：ok，尝试次数：2] - 96418.https://www.dormanproducts.com/p-85749-955-2365.aspx
[剩余数量：996] - [当前时间：14:21:48]

[状态：ok，尝试次数：1] - 96453.https://www.dormanproducts.com/p-52858-955-1528.aspx
[剩余数量：995] - [当前时间：14:21:49]

[状态：ok，尝试次数：1] - 96433.https://www.dormanproducts.com/p-103172-955-2456.aspx
[剩余数量：994] - [当前时间：14:21:49]

[状态：ok，尝试次数：1] - 96436.https://www.dormanproducts.com/p-33392-955-441.aspx
[剩余数量：993] - [当前时间：14:21:49]

[状态：ok，尝试次数：1] - 96437.https://www.dormanproducts.com/p-11689-955-310.aspx
[剩余数量：992] - [当前时间：14:21:50]

[状态：ok，尝试次数：1] - 96443.https://www.dormanproducts